# Phylogenetic analysis of variance (PANOVA)

## Data preparation

### Loading packages

In [ ]:
library("stringr")
library("phytools")
library("RRPP")

### Loading data

In [ ]:
args<-commandArgs(trailingOnly = TRUE)

In [ ]:
df<-read.csv("../../raw_data/Trait_data_Squaliformes_Fossil.tsv", sep="\t") # omit sep ="\t" for .csv files
phy<-read.tree("../../raw_data/Squaliformes_extant.tree")

### Cleaning data

In [ ]:
df$Species<-str_replace(df$Species, " ", "_")
states<-df
states_traits<-states[!states$Species %in% setdiff(states$Species, phy$tip.label),]

### Preparing Data

In [ ]:
categorical_trait<-states_traits[,as.numeric(args[1])]
names(categorical_trait)<-states_traits$Species
continuous_trait<-log(states_traits$Body.size)
names(continuous_trait)<-states_traits$Species

## Running PANOVA (simPANOVA)

In [ ]:
phanov_sim <- phylANOVA(phy, categorical_trait, continuous_trait, nsim=10000, posthoc=TRUE, p.adj="bonferroni")

## Running PANOVA (RRPP PANOVA)

In [ ]:
GLS_PANOVA_RRPP <- lm.rrpp(continuous_trait ~ categorical_trait, 
                  print.progress = FALSE,
                  Cov = vcv(phy),
                  turbo = FALSE, verbose = TRUE)

### Saving data

In [ ]:
saveRDS(phanov_sim, paste("panova_sim_", args[2], ".rds", sep = ""))

In [ ]:
saveRDS(anova(GLS_PANOVA_RRPP), paste("panova_RRPP_", args[2], ".rds", sep = ""))